In [1]:
import pandas as pd
import gc

In [2]:
from sklearn.preprocessing import StandardScaler

scalerEnginer = StandardScaler()

In [3]:
movies_df = pd.read_csv('./data/ml-latest/movies.csv', usecols=['movieId', 'title', 'genres'])
rating_df = pd.read_csv('./data/ml-latest/ratings.csv')

rating_df['rating'] = scalerEnginer.fit_transform(rating_df[['rating']])

gc.collect()

0

In [4]:
# Eliminar duplicados en el DataFrame de películas
movies_df.drop_duplicates(inplace=True)

# Obtener las películas sin calificaciones
movies_without_ratings = movies_df[~movies_df['movieId'].isin(rating_df['movieId'])]
movies_without_ratings_rows = movies_without_ratings[['movieId', 'title']]
print('MOVIES WITHOUT RATINGS:', movies_without_ratings_rows.shape)

# Filtrar películas con calificaciones
movies_cleaned = movies_df[~movies_df['movieId'].isin(movies_without_ratings_rows['movieId'])]

# Calcular el promedio de las calificaciones por película
rating_avg = rating_df.groupby('movieId')['rating'].mean().reset_index(name='rating_avg')

# Contar el número de calificaciones por película
rating_count = rating_df.groupby('movieId').size().reset_index(name='rating_count')

# Unir los DataFrames de películas limpias con el promedio y el conteo de calificaciones
movies_cleaned = pd.merge(movies_cleaned, rating_count, on='movieId', how='left')
movies_cleaned = pd.merge(movies_cleaned, rating_avg, on='movieId', how='left')

# Imprimir la información del DataFrame de películas limpias
print('CLEANED MOVIES CSV:', movies_df.shape, '->', movies_cleaned.shape)

gc.collect()

MOVIES WITHOUT RATINGS: (4209, 2)
CLEANED MOVIES CSV: (58098, 3) -> (53889, 5)


0

In [5]:
links_df = pd.read_csv('./data/ml-latest/links.csv', usecols=['movieId', 'tmdbId'], dtype={'tmdbId': str})
links_df.head(3)

,movieId,tmdbId
0,1,862
1,2,8844
2,3,15602


In [6]:
movies_cleaned = pd.merge(movies_cleaned, links_df, on='movieId')

In [7]:
movies_cleaned.head()

,movieId,title,genres,rating_count,rating_avg,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,68469,0.334040,862
1,2,Jumanji (1995),Adventure|Children|Fantasy,27143,-0.266199,8844
2,3,Grumpier Old Men (1995),Comedy|Romance,15585,-0.334283,15602
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2989,-0.615092,31357
4,5,Father of the Bride Part II (1995),Comedy,15474,-0.424957,11862
